## Simulating melts of polymer chains

In [1]:
import warnings
warnings.filterwarnings('ignore')
import hoomd
import gsd
import gsd.hoomd
from hoomd.md import nlist

import unyt as u
import glob
import mbuild as mb
import matplotlib.pyplot as plt
import numpy as np
import os

import flowermd
from flowermd.utils import get_target_box_mass_density
from flowermd.base import Simulation, Molecule
from flowermd.library import FF_from_file
from flowermd.base.system import Pack

from polymer_dictionary import polymer_dictionary

# Rerun with the original seed first

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/pkg_resources/__init__.py:3146: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)



Support for writing out LAMMPS data files will be removed
in mbuild 1.0.
See GMSO (https://github.com/mosdef-hub/gmso/tree/main/gmso/formats/lammpsdata) for
continued support for LAMMPS.



In [2]:
# Updates atom names to fit with hoomd conventions
def espaloma_mol(file_path):
     mol = mb.load(file_path)
     for p in mol.particles():
           p.name = f"_{p.name}"
     return mol

## Single molecule sim:

In [3]:
key_list = sorted(list(polymer_dictionary.keys())) # Aligning dictionary and path to aid automation
path = os.getcwd() # Non-user specific path
molecule_list = sorted(glob.glob(path+"/mol2/*.mol2"))
molecule_list.pop(10) # Removing polymer not tested
print(key_list)

['PCPDTFBT_C11_BO', 'PCPDTFBT_C1_BO', 'PCPDTFBT_C3_BO', 'PCPDTFBT_C4_BO', 'PCPDTFBT_C5_BO', 'PCPDTPT_HD', 'PCPDTPT_ODD', 'PCPDTPT_eneODD', 'PCPDTPT_nC16', 'PCPDT_PT_eneHD', 'PIDTBT_nC16', 'PIDTCPDT_C11BO', 'PIDTFBT_C11_BO']


In [4]:
mol2_path = os.getcwd() + "/mol2/"
xml_path = os.getcwd() + "/xml/"

In [4]:
system_file = mol2_path + "alternate_PCPDTPT_nC16.mol2"
ff_filepath = xml_path + "alternate_PCPDTPT_nC16.xml"

espmol = espaloma_mol(system_file)
molecule = Molecule(num_mols=1, compound=espmol)

molff = FF_from_file(ff_filepath)
system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,packing_expand_factor=5)
system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


No charged group detected, skipping electrostatics.


In [5]:
system.hoomd_snapshot
hoomd_forces = system.hoomd_forcefield
hoomd_forces
lj_force = hoomd_forces[3]
cpu = hoomd.device.CPU()
sim = Simulation.from_system(system=system, gsd_write_freq=1, log_write_freq=1, device=cpu, gsd_file_name="alternate_PCPDTPT_nC16_melt.gsd",log_file_name="alternate_PCPDTPT_nC16_melt.txt")

Initializing simulation state from a gsd.hoomd.Frame.


In [6]:
sim.run_NVT(n_steps=1, kT=5.0, tau_kt=1.0)
sim.flush_writers()

In [24]:
sim.reference_values

{'energy': unyt_quantity(1.046, 'kJ/mol'),
 'length': unyt_quantity(0.35635949, 'nm'),
 'mass': unyt_quantity(32.06, 'amu')}

## Running sims of every polymer in the Danielsen et al. paper

In [10]:
for i in range(len(key_list)):
    system_file = mol2_path + key_list[i] + ".mol2"
    ff_filepath = xml_path + key_list[i] + ".xml"
    num_mers = "_10mer" # The number of monomers being simulated is determined by the mol2, this is for naming the resulting gsd and .txt files
    espmol = espaloma_mol(system_file)
    molecule = Molecule(num_mols=1, compound=espmol)
    
    molff = FF_from_file(ff_filepath)
    system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,packing_expand_factor=5)
    system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)
    system.hoomd_snapshot
    hoomd_forces = system.hoomd_forcefield
    hoomd_forces
    lj_force = hoomd_forces[3]
    cpu = hoomd.device.CPU()
    sim = Simulation.from_system(system=system, gsd_write_freq=1000, log_write_freq=1000, device=cpu, gsd_file_name=key_list[i]+num_mers+".gsd",log_file_name=key_list[i]+num_mers+".txt")
    sim.run_NVT(n_steps=1e6, kT=5.0, tau_kt=1.0)
    sim.flush_writers()

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 1000000; TPS: 2662.75; ETA: 6.3 minutes
Step 2000 of 1000000; TPS: 4123.88; ETA: 4.0 minutes
Step 3000 of 1000000; TPS: 5034.51; ETA: 3.3 minutes
Step 4000 of 1000000; TPS: 5654.01; ETA: 2.9 minutes
Step 5000 of 1000000; TPS: 6104.0; ETA: 2.7 minutes
Step 6000 of 1000000; TPS: 6501.2; ETA: 2.5 minutes
Step 7000 of 1000000; TPS: 6768.68; ETA: 2.4 minutes
Step 8000 of 1000000; TPS: 6914.27; ETA: 2.4 minutes
Step 9000 of 1000000; TPS: 7144.75; ETA: 2.3 minutes
Step 10000 of 1000000; TPS: 7352.62; ETA: 2.2 minutes
Step 11000 of 1000000; TPS: 7525.87; ETA: 2.2 minutes
Step 12000 of 1000000; TPS: 7595.86; ETA: 2.2 minutes
Step 13000 of 1000000; TPS: 7684.97; ETA: 2.1 minutes
Step 14000 of 1000000; TPS: 7792.14; ETA: 2.1 minutes
Step 15000 of 1000000; TPS: 7913.45; ETA: 2.1 minutes
Step 16000 of 1000000; TPS: 8032.12; ETA: 2.0 minutes
Step 17000 of 1000000; TPS: 8137.42; ETA:

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 1000000; TPS: 3403.59; ETA: 4.9 minutes
Step 2000 of 1000000; TPS: 5294.52; ETA: 3.1 minutes
Step 3000 of 1000000; TPS: 6402.54; ETA: 2.6 minutes
Step 4000 of 1000000; TPS: 7220.33; ETA: 2.3 minutes
Step 5000 of 1000000; TPS: 7819.98; ETA: 2.1 minutes
Step 6000 of 1000000; TPS: 8282.81; ETA: 2.0 minutes
Step 7000 of 1000000; TPS: 8629.74; ETA: 1.9 minutes
Step 8000 of 1000000; TPS: 8941.82; ETA: 1.8 minutes
Step 9000 of 1000000; TPS: 9138.45; ETA: 1.8 minutes
Step 10000 of 1000000; TPS: 9339.31; ETA: 1.8 minutes
Step 11000 of 1000000; TPS: 9553.53; ETA: 1.7 minutes
Step 12000 of 1000000; TPS: 9739.52; ETA: 1.7 minutes
Step 13000 of 1000000; TPS: 9884.21; ETA: 1.7 minutes
Step 14000 of 1000000; TPS: 9979.09; ETA: 1.6 minutes
Step 15000 of 1000000; TPS: 10111.24; ETA: 1.6 minutes
Step 16000 of 1000000; TPS: 10238.39; ETA: 1.6 minutes
Step 17000 of 1000000; TPS: 10333.12;

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


...

No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 1000000; TPS: 3521.4; ETA: 4.7 minutes
Step 2000 of 1000000; TPS: 5625.06; ETA: 3.0 minutes
Step 3000 of 1000000; TPS: 7116.69; ETA: 2.3 minutes
Step 4000 of 1000000; TPS: 8170.42; ETA: 2.0 minutes
Step 5000 of 1000000; TPS: 8952.71; ETA: 1.9 minutes
Step 6000 of 1000000; TPS: 9614.46; ETA: 1.7 minutes
Step 7000 of 1000000; TPS: 10148.43; ETA: 1.6 minutes
Step 8000 of 1000000; TPS: 10567.74; ETA: 1.6 minutes
Step 9000 of 1000000; TPS: 10939.83; ETA: 1.5 minutes
Step 10000 of 1000000; TPS: 11208.07; ETA: 1.5 minutes
Step 11000 of 1000000; TPS: 11483.01; ETA: 1.4 minutes
Step 12000 of 1000000; TPS: 11677.53; ETA: 1.4 minutes
Step 13000 of 1000000; TPS: 11891.56; ETA: 1.4 minutes
Step 14000 of 1000000; TPS: 12079.15; ETA: 1.4 minutes
Step 15000 of 1000000; TPS: 12211.82; ETA: 1.3 minutes
Step 16000 of 1000000; TPS: 12304.41; ETA: 1.3 minutes
Step 17000 of 1000000; TPS: 12

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 1000000; TPS: 2701.96; ETA: 6.2 minutes
Step 2000 of 1000000; TPS: 3911.55; ETA: 4.3 minutes
Step 3000 of 1000000; TPS: 4506.9; ETA: 3.7 minutes
Step 4000 of 1000000; TPS: 4926.96; ETA: 3.4 minutes
Step 5000 of 1000000; TPS: 5269.65; ETA: 3.1 minutes
Step 6000 of 1000000; TPS: 5545.01; ETA: 3.0 minutes
Step 7000 of 1000000; TPS: 5802.33; ETA: 2.9 minutes
Step 8000 of 1000000; TPS: 6009.55; ETA: 2.8 minutes
Step 9000 of 1000000; TPS: 6173.39; ETA: 2.7 minutes
Step 10000 of 1000000; TPS: 6315.15; ETA: 2.6 minutes
Step 11000 of 1000000; TPS: 6430.54; ETA: 2.6 minutes
Step 12000 of 1000000; TPS: 6554.12; ETA: 2.5 minutes
Step 13000 of 1000000; TPS: 6651.7; ETA: 2.5 minutes
Step 14000 of 1000000; TPS: 6648.96; ETA: 2.5 minutes
Step 15000 of 1000000; TPS: 6672.89; ETA: 2.5 minutes
Step 16000 of 1000000; TPS: 6717.25; ETA: 2.4 minutes
Step 17000 of 1000000; TPS: 6790.23; ETA:

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 1000000; TPS: 2076.1; ETA: 8.0 minutes
Step 2000 of 1000000; TPS: 2815.89; ETA: 5.9 minutes
Step 3000 of 1000000; TPS: 3159.17; ETA: 5.3 minutes
Step 4000 of 1000000; TPS: 3228.07; ETA: 5.1 minutes
Step 5000 of 1000000; TPS: 3420.23; ETA: 4.8 minutes
Step 6000 of 1000000; TPS: 3583.33; ETA: 4.6 minutes
Step 7000 of 1000000; TPS: 3680.63; ETA: 4.5 minutes
Step 8000 of 1000000; TPS: 3774.22; ETA: 4.4 minutes
Step 9000 of 1000000; TPS: 3857.63; ETA: 4.3 minutes
Step 10000 of 1000000; TPS: 3944.72; ETA: 4.2 minutes
Step 11000 of 1000000; TPS: 4021.27; ETA: 4.1 minutes
Step 12000 of 1000000; TPS: 4091.91; ETA: 4.0 minutes
Step 13000 of 1000000; TPS: 4151.43; ETA: 4.0 minutes
Step 14000 of 1000000; TPS: 4197.3; ETA: 3.9 minutes
Step 15000 of 1000000; TPS: 4257.34; ETA: 3.9 minutes
Step 16000 of 1000000; TPS: 4301.97; ETA: 3.8 minutes
Step 17000 of 1000000; TPS: 4352.44; ETA: